In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5


--2024-02-17 08:42:56--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 74.125.196.207, 74.125.134.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   236MB/s    in 0.4s    

2024-02-17 08:42:56 (236 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [19]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [22]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

pre_trained_model = InceptionV3(input_shape=(150,150,3), include_top= False, weights = None)
pre_trained_model.load_weights(local_weights_file)

In [23]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_564 (Conv2D)         (None, 74, 74, 32)           864       ['input_7[0][0]']             
                                                                                                  
 batch_normalization_564 (B  (None, 74, 74, 32)           96        ['conv2d_564[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_564 (Activation  (None, 74, 74, 32)           0         ['batch_normalizati

In [27]:
for layer in pre_trained_model.layers:
  layer.trainable= False

In [28]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_564 (Conv2D)         (None, 74, 74, 32)           864       ['input_7[0][0]']             
                                                                                                  
 batch_normalization_564 (B  (None, 74, 74, 32)           96        ['conv2d_564[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_564 (Activation  (None, 74, 74, 32)           0         ['batch_normalizati

In [29]:
last_layer = pre_trained_model.get_layer('mixed7') # có nghĩa là giữ lại từ đầu đến đoạn cắt xóa từ đoạn cắt đến hết ( đoạn cắt được gọi là mixed7)

In [30]:
last_output = last_layer.output

In [31]:
last_output

<KerasTensor: shape=(None, 7, 7, 768) dtype=float32 (created by layer 'mixed7')>

In [35]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output) # truyền model đã train vào lớp flatten()
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)

model = Model( pre_trained_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])


In [36]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_564 (Conv2D)         (None, 74, 74, 32)           864       ['input_7[0][0]']             
                                                                                                  
 batch_normalization_564 (B  (None, 74, 74, 32)           96        ['conv2d_564[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_564 (Activation  (None, 74, 74, 32)           0         ['batch_normalization_56

In [37]:
!gdown 1ALsADt1blTFYLZMG7wZHZJoTvYwK42XO
!gdown 1K4vwjbZobaEyMO0UDbe9-OgPEfApa2zl

Downloading...
From (original): https://drive.google.com/uc?id=1ALsADt1blTFYLZMG7wZHZJoTvYwK42XO
From (redirected): https://drive.google.com/uc?id=1ALsADt1blTFYLZMG7wZHZJoTvYwK42XO&confirm=t&uuid=9a5ce1ba-baa1-4872-94de-9b0e66e4e807
To: /content/drive/MyDrive/Model Inception V3/horse-or-human.zip
100% 150M/150M [00:04<00:00, 36.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1K4vwjbZobaEyMO0UDbe9-OgPEfApa2zl
To: /content/drive/MyDrive/Model Inception V3/validation-horse-or-human.zip
100% 11.6M/11.6M [00:00<00:00, 17.8MB/s]


In [38]:
!unzip horse-or-human.zip

!unzip -q validation-horse-or-human.zip

train_horse_dir = './train/horses'
train_human_dir = './train/humans'
valid_horse_dir = './validation/horses'
valid_human_dir = './validation/humans'

train_folder = './train'
valid_folder = './validation'

Archive:  horse-or-human.zip
   creating: train/
  inflating: __MACOSX/._train        
   creating: train/humans/
  inflating: __MACOSX/train/._humans  
   creating: train/horses/
  inflating: __MACOSX/train/._horses  
  inflating: train/humans/human17-22.png  
  inflating: __MACOSX/train/humans/._human17-22.png  
  inflating: train/humans/human10-17.png  
  inflating: __MACOSX/train/humans/._human10-17.png  
  inflating: train/humans/human10-03.png  
  inflating: __MACOSX/train/humans/._human10-03.png  
  inflating: train/humans/human07-27.png  
  inflating: __MACOSX/train/humans/._human07-27.png  
  inflating: train/humans/human09-22.png  
  inflating: __MACOSX/train/humans/._human09-22.png  
  inflating: train/humans/human05-22.png  
  inflating: __MACOSX/train/humans/._human05-22.png  
  inflating: train/humans/human02-03.png  
  inflating: __MACOSX/train/humans/._human02-03.png  
  inflating: train/humans/human02-17.png  
  inflating: __MACOSX/train/humans/._human02-17.png  
  inf

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [40]:
train_generator = ImageDataGenerator(rescale=1/255)
valid_generator = ImageDataGenerator(rescale=1/255)

In [41]:
generated_train_data = train_generator.flow_from_directory(
    train_folder,
    target_size= (150, 150),
    batch_size=64,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [42]:
generated_valid_data = valid_generator.flow_from_directory(
    valid_folder,
    target_size= (150, 150),
    batch_size=64,
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [43]:
model.fit(generated_train_data, epochs = 10, validation_data = generated_valid_data)

Epoch 1/10
17/17 [==============================] - 89s 5s/step - loss: 0.1100 - accuracy: 0.9484 - val_loss: 0.0153 - val_accuracy: 0.9961
Epoch 2/10
17/17 [==============================] - 82s 5s/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0169 - val_accuracy: 0.9922
Epoch 3/10
 3/17 [====>.........................] - ETA: 54s - loss: 6.2992e-04 - accuracy: 1.0000

KeyboardInterrupt: 